In [54]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import __future__
from random import random
from scipy import signal
from math import log10
import os
%matplotlib inline

<h2>Função de Correção</h2>
<p>A impressão de imagens com opencv utilizando valores flutuantes deve ser feita entre o range [0.0, 1.0]</p>

In [2]:
fix = lambda img: cv2.normalize(img, None, 0.0, 1.0, cv2.NORM_MINMAX)

<h2>Funções de Ruídos</h2>
<p>As funções utilizadas para adicionar ruídos são, respectivamente:</p>
<ul>
    <li>Sal e Pimenta</li>
    <li>Alongamento de Contraste</li>
    <li>Adição de Ruído Gaussiano</li>
    <li>Compressão JPEG</li>
    <li>Ruído Speckle</li>
    <li>Embaçamento</li>
</ul>

In [14]:
def saltPepper(img, prob = 0.05):
    pepper = prob
    salt = 1-prob
    output = np.zeros(img.shape, np.float)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rand = random()
            if rand < pepper:
                intensity = [0.,0.,0.]
            elif rand > salt:
                intensity = [255.,255.,255.]
            else:
                intensity = [img[i][j][0], img[i][j][1], img[i][j][2]]
            output[i][j] = intensity
    return output

In [15]:
contrastStretching = lambda img: (img-img.min())/(img.max()-img.min())*255.

In [16]:
def additiveGaussianNoise(img, mean = 0, var = 0.1):
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,img.shape)
    gauss = gauss.reshape(img.shape)
    return img + gauss

In [100]:
def jpegCompression(img):
    encode_param = [cv2.IMWRITE_JPEG_QUALITY, 90]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    return cv2.imdecode(encimg, 3).astype(np.float)

In [18]:
blurring = lambda img, width, height : cv2.blur(img, (width, height))

In [19]:
def speckleNoise(img, mean = 0, var = 0.1):
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,img.shape)
    gauss = gauss.reshape(img.shape)
    return img + img*gauss

<h2>Funções de Qualidade</h2>
<p>Implementação das funções de qualidade que serão utilizadas:</p>
<ul>
    <li>Universal Quality Index (a função proposta no artigo)</li>
    <li>MSE: Erro médio quadrático (Root será opcional)</li>
    <li>PSNR: Relação sinal-ruído de pico</li>
</ul>

In [22]:
img = cv2.imread('pictures/lena/lena.png').astype(np.float)

<h3>Universal Quality Index</h3>
<img src="uqi3.png">
<p>A implementação desse índice pode ser vista em 3 partes:</p>
<ul>
    <li>Correlação entre as Imagens</li>
    <li>Semelhança da Luminescência</li>
    <li>Semelhança de Contraste das Imagens</li>
</ul>
<p>Equação Final:</p>
<img src="uqi.png">

<p>A função numCorrelation serve para calcular o numerador da função de correlação vista anteriormente de forma manual para uma imagem RGB, onde é retirado a média das cores no pixel (i,j) e é calculado o somatório do produto desses valores subtraído da média para cada uma das imagens</p>

In [24]:
def numCorrelation(a, b):
    a_mean = a.mean()
    b_mean = b.mean()
    num = 0
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            local_mean_a = a[i][j].mean()
            local_mean_b = b[i][j].mean()
            local_calc = (local_mean_a-a_mean)*(local_mean_b-b_mean)
            num += local_calc
    return num/(a.shape[0]*a.shape[1]-1) 

In [44]:
def quality(img, noise_img):
    x_mean = img.mean()
    y_mean = noise_img.mean()
    x_std = img.std()
    y_std = noise_img.std()
    xy_std = numCorrelation(img,noise_img)
    top = 4.*xy_std*x_mean*y_mean
    bot = (x_std**2.+y_std**2.)*(x_mean**2.+y_mean**2.)
    return top/bot

In [45]:
mse = lambda img, noise_img, root: ((img - noise_img)**2).mean() if not root else ((img - noise_img)**2).mean()**0.5

In [51]:
psnr = lambda img, noise_img: log10(255.0/mse(img, noise_img, True))

<h2>Inserindo Ruídos e Extraindo Métricas</h2>
<p>Inicialmente serão inserido ruídos nas imagens iniciais e os resultados das imagens antes e após filtros serão impressos, junto com seus índices de qualidade</p>

In [105]:
BASE_PATH = "./pictures/base/"
for base_img in os.listdir(BASE_PATH):
    IMG_PATH = BASE_PATH+base_img
    img = cv2.imread(IMG_PATH).astype(np.float)
    # Get images with noise
    sp_img = saltPepper(img, prob=0.02)
    cs_img = contrastStretching(img)
    gauss_img = additiveGaussianNoise(img, var=100)
    jpeg_img = jpegCompression(img)
    blur_img = blurring(img, 10, 10)
    speckle_img = speckleNoise(img, var=0.01)
    # Show Images
    fig = plt.figure(figsize=(13,10))

    ax = plt.subplot(231)
    ax.set_title("Salt And Pepper")
    ax.axis("off")
    ax.imshow(fix(sp_img))

    ax = plt.subplot(232)
    ax.set_title("Contrast Stretching")
    ax.axis("off")
    ax.imshow(fix(cs_img))
    
    ax = plt.subplot(233)
    ax.set_title("Gaussian Noise")
    ax.axis("off")
    ax.imshow(fix(gauss_img))
    
    ax = plt.subplot(234)
    ax.set_title("JPEG Compression")
    ax.axis("off")
    ax.imshow(fix(jpeg_img))
    
    ax = plt.subplot(235)
    ax.set_title("Mean Blur")
    ax.axis("off")
    ax.imshow(fix(blur_img))
    
    ax = plt.subplot(236)
    ax.set_title("Speckle Noise")
    ax.axis("off")
    ax.imshow(fix(speckle_img))
    plt.show()

AttributeError: 'Figure' object has no attribute 'set_title'

<Figure size 936x720 with 0 Axes>